# EDA216 - Projekt


## Preparation

Load the Jupyter SQL extension, create a new empty database and load database by evaluating the next cells:

In [ ]:
%load_ext sql

Ladda in och skapa en test-databas med påhittade värden

In [ ]:
%%bash
sqlite3 src/notebook.db < src/cookie.sql


In [ ]:
%%bash
sqlite3 src/notebook.db < src/orders.sql

In [ ]:
%%bash
sqlite3 src/notebook.db < src/pallets.sql


In [ ]:
%%bash
sqlite3 src/notebook.db < src/ingredients.sql

In [ ]:
%sql sqlite:///src/notebook.db

In [ ]:
%sql PRAGMA foreign_keys=ON;

Raderar databasen. Använd ej

In [ ]:
%%bash
rm -f src/notebook.db

# Sql commands - Simple

In [ ]:
%%sql
-- See all customers --
SELECT * 
FROM Customer

In [ ]:
%%sql
-- See all cookies --
SELECT *
FROM Cookie

In [ ]:
%%sql
-- See all orders --
SELEcT *
FROM Order_Bill

# FOR CONNECTING PALLETS TO AN ORDER

In [ ]:
%%sql
-- Adding a new order --
INSERT INTO Order_Bill (order_id, /* Order_id must be omitted later */ customer_name, delivery_date)
VALUES (1001, /* 1001  must be omitted later */ "Partykakor AB", "2017-08-12");


In [ ]:
%%sql
INSERT INTO OrderItems (order_id, cookie_name, nbrPallet)
VALUES (1001, /* Must be omitted later */ "Berliner", 8),
(1001, /* Must be omitted later */ "Nut ring", 4);

In [ ]:
%%sql
-- Show Orders order items for a specific order --
Select cookie_name, nbrPallet
FROM Order_Bill INNER JOIN OrderItems ON Order_Bill.order_id = OrderItems.order_id
WHERE Order_Bill.order_id = 1001

In [ ]:
%%sql
-- Show all pallets --
SELECT * 
FROM Pallet 
--WHERE order_id IS NULL AND is_blocked IS NOT 1 --uncomment for only unconnected pallets 
ORDER BY order_id DESC




In [ ]:
%%sql
-- Connect the oldest produced Berliner to Order id 101 (5 pallets of Berliner)--
UPDATE Pallet
set order_id = 103
WHERE cookie_name = 'Berliner' AND order_id IS NULL AND is_blocked IS NOT 1
order by production_date desc
limit 1

# Stuff - Pallets

In [ ]:
%%sql
-- A list of pallet by cookie type between two dates
SELECT *
FROM Pallet
WHERE cookie_name = 'Berliner' AND
production_date BETWEEN "2017-01-01" AND "2017-02-23"

In [ ]:
%%sql
-- Create a pallet, connected to a unspecific order --
INSERT INTO Pallet (cookie_name, order_id, production_date, location, is_blocked)
VALUES ('Nut ring', NULL ,'2017-02-12', 'asdghkskajd', NULL);



In [ ]:
%%sql
-- Show all entries of Berliner--
SELECT * 
FROM Pallet
WHERE cookie_name = 'Berliner'

In [ ]:
%%sql
-- Unlock all panels
UPDATE Pallet
set is_blocked = 0

In [ ]:
%%sql
-- set delivered
UPDATE Pallet
set location = 'Delivered'
WHERE pallet_id = 45

In [ ]:
%%sql
-- Show number of Berliner that are unconnected that are not blocked--
SELECT COUNT(*) 
FROM Pallet
WHERE cookie_name = 'Berliner' AND order_id IS NULL AND is_blocked = 0

In [ ]:
%%sql
-- Show number of Berliner left to produce for roder 103 --
SELECT COUNT(*) - (SELECT COUNT(*) FROM Pallet WHERE order_id = 103 AND cookie_name = 'Berliner' AND is_blocked IS NOT 1) as Val
FROM Pallet
WHERE Pallet.cookie_name = 'Berliner' AND Pallet.order_id IS NULL AND is_blocked = 0



# Uppdatera lagret med råvaror

In [ ]:
%%sql
SELECT * 
FROM Ingredient

In [ ]:
%%sql
-- Fill the database with values
UPDATE Ingredient
set amount = 500000, refill_date = "2017-03-19" --DATE()
--WHERE ingredient_name = 'Butter'

In [ ]:
%%sql
-- Get ingredients needed for a cookie
SELECT ingredient_name, amount*54 as 'amount for pallet'
FROM RecipeItems
Where cookie_name = 'Berliner'

In [ ]:
%%sql
-- Reduce the ingredient stock version 1
-- TODO: replace 1 with 54*amount of a ingredient
UPDATE Ingredient
set amount = amount - 1
WHERE ingredient_name = 'Butter' 

In [ ]:
%%sql
-- Reduce ingredient stock version 2
UPDATE Ingredient
       set amount = CASE
            WHEN ingredient_name = 'Butter' THEN amount - 54*(SELECT amount from RecipeItems WHERE cookie_name = 'Berliner' AND ingredient_name = 'Butter')
            WHEN ingredient_name = 'Eggs' THEN amount - 54*(SELECT amount from RecipeItems WHERE cookie_name = 'Berliner' AND ingredient_name = 'Eggs')
            WHEN ingredient_name = 'Chocolate' THEN amount - 54*(SELECT amount from RecipeItems WHERE cookie_name = 'Berliner' AND ingredient_name = 'Chocolate')
            ELSE amount END

# Oh no, the cookie Berliner is infected by a terrible worm! Ach nein!

In [ ]:
%%sql
-- Block all pallets between two dates
UPDATE Pallet
    set is_blocked = CASE WHEN cookie_name = 'Berliner' AND location IS NOT 'Delivered' AND production_date BETWEEN '2016-01-01' AND '2018-01-01'  THEN 1
    ELSE is_blocked END


# Loading Order
Customer 1 name, adress, # of pallets of cookie a, # pallets of cookie b

# Track a pallet
see cookie_type, location, delivered, to whom?

# Listor att exportera

In [ ]:
%%sql
-- Pallar producerade mellan två datum
SELECT * 
FROM PALLET
WHERE production_date BETWEEN "2016-01-01" AND "2017-02-23"

In [ ]:
%%sql
-- Blockerade pallar
SELECT pallet_id
FROM PALLET
WHERE is_blocked = 1